In [43]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
df = pd.read_csv('../PlayTennis.csv')
df

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [93]:
X = df.drop(['Play Tennis'], axis = 1)
y = df['Play Tennis']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7)
map_yes, map_no, n_yes, n_no = compute_likelihood(X, y)

In [94]:
arr = classify(X_test, y_test, map_yes, map_no, n_yes, n_no)
print(y_test)
print(arr)

1      No
6     Yes
11    Yes
0      No
12    Yes
Name: Play Tennis, dtype: object
['no', 'yes', 'yes', 'no', 'yes']


In [64]:
def compute_likelihood(X, y):
    b = 0
    map_yes = {}
    map_no = {}
    n_yes = 0
    n_no = 0
    for col in X.columns:
        b = b + 1
        for a, val in enumerate(X[col]):
            if (y[a] == "Yes"):
                if (b == 1):
                    n_yes = n_yes + 1
                new_val = map_yes.get(val, 0) + 1
                map_yes.update({val : new_val})
            else:
                if (b == 1):
                    n_no = n_no + 1
                new_val = map_no.get(val, 0) + 1
                map_no.update({val : new_val})
    return map_yes, map_no, n_yes, n_no

In [86]:
def posterior_yes(instance, map_yes, n_yes, n_no):
    prod = (n_yes) / (n_yes + n_no)
    for keys in instance:
        prod = prod * (map_yes.get(keys, 0) / n_yes)
    return prod

In [87]:
def posterior_no(instance, map_no, n_yes, n_no):
    prod = (n_no) / (n_yes + n_no)
    for keys in instance:
        prod = prod * (map_no.get(keys, 0) / n_no)
    return prod

In [88]:
def classify(X, y, map_yes, map_no, n_yes, n_no):
    outputs = []
    for a, row in X.iterrows():
        if (posterior_yes(row, map_yes, n_yes, n_no) > posterior_no(row, map_no, n_yes, n_no)):
            outputs.append("yes")
        else:
            outputs.append("no")
    return outputs